In [2]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(dplyr))
library(magrittr)

In [13]:
## closely followed PRS tutorial referenced in github
## filtering by heterozygosity and mismatched variants
het = fread("bfiles/ALL_COVID.QC.het")
valid = het[F<=mean(F)+3*sd(F) & F>=mean(F)-3*sd(F)] 

bim = fread("bfiles/ALL_COVID.bim")
colnames(bim) = c("CHR", "SNP", "CM", "BP", "B.A1", "B.A2")
qc = read.table("bfiles/ALL_COVID.QC.snplist", header = F, stringsAsFactors = F)
gwas = fread("COVID19_HGI_A2_filtered.txt")
names(gwas)[2] = "BP"

info = merge(bim, gwas, by=c("SNP", "CHR", "BP")) %>%
    .[SNP %in% qc[,"V1"]]
names(info)[7] = "A2"
names(info)[8] = "A1"

complement = function(x){
    switch (x,
        "A" = "T",
        "C" = "G",
        "T" = "A",
        "G" = "C",
        return(NA)
    )
} 

info.match = info[A1 == B.A1 & A2 == B.A2, SNP]

com.snps = info[sapply(B.A1, complement) == A1 &
                    sapply(B.A2, complement) == A2, SNP]

bim[SNP %in% com.snps, c("B.A1", "B.A2") :=
        list(sapply(B.A1, complement),
            sapply(B.A2, complement))]

recode.snps = info[B.A1==A2 & B.A2==A1, SNP]

bim[SNP %in% recode.snps, c("B.A1", "B.A2") :=
        list(B.A2, B.A1)]


com.recode = info[sapply(B.A1, complement) == A2 &
                    sapply(B.A2, complement) == A1, SNP]

bim[SNP %in% com.recode, c("B.A1", "B.A2") :=
        list(sapply(B.A2, complement),
            sapply(B.A1, complement))]
mismatch = bim[!(SNP %in% info.match |
                    SNP %in% com.snps |
                    SNP %in% recode.snps |
                    SNP %in% com.recode), SNP]
write.table(mismatch, "bfiles/ALL_COVID.mismatch", quote=F, row.names=F, col.names=F)
fwrite(bim[,c("SNP", "B.A1")], "bfiles/ALL_COVID.a1", col.names=F, sep="\t")
fwrite(valid[,c("FID","IID")], "bfiles/ALL_COVID.valid.sample", sep="\t") 

In [163]:
## make validation and training sets
p.threshold = c("0.000001", "0.00001", "0.001","0.05","0.1","0.2","0.3","0.4","0.5")

phenotype = read.table("/frazer01/home/jennifer/private/cse284_project/analysis/boogie/Arvados-Blood-Types-and-Ethnicity/AnalyzeBloodByEthnicity/433-1000Genomes/outputBlood.csv", sep=",",header=T)

pcs = read.table("ALL_COVID.eigenvec", header=F)

colnames(pcs) = c("FID", "IID", paste0("PC",1:6))

names(phenotype)[1] = "FID"
phenotype$IID = phenotype$FID
phenotype$Blood.Type[phenotype$Blood.Type != "O"] = 0
phenotype$Blood.Type[phenotype$Blood.Type == "O"] = 1

training = phenotype[1:327,]
validation = phenotype[328:nrow(phenotype),]

pheno = merge(training, pcs, by=c("FID","IID"))

null.r2 = summary(lm(Blood.Type~., data=pheno[,!colnames(pheno)%in%c("FID","IID")]))$r.squared
prs.result = NULL
for(i in p.threshold){
    pheno.prs = merge(pheno, 
                        read.table(paste0("ALL_COVID.",i,".profile"), header=T)[,c("FID","IID", "SCORE")],
                        by=c("FID", "IID"))
    model = summary(lm(Blood.Type~., data=pheno.prs[,!colnames(pheno.prs)%in%c("FID","IID")]))
    model.r2 = model$r.squared
    prs.r2 = model.r2-null.r2
    prs.coef = model$coeff["SCORE",]
    prs.result = rbind(prs.result, 
        data.frame(Threshold=i, R2=prs.r2, 
                    P=as.numeric(prs.coef[4]), 
                    BETA=as.numeric(prs.coef[1]),
                    SE=as.numeric(prs.coef[2])))
}

fwrite(training, "~/cse284/bfiles/training_set.txt",
      sep='\t', quote=F, row.names=F)

fwrite(validation, "~/cse284/bfiles/validation_set.txt",
      sep='\t', quote=F, row.names=F)

[1] 327

[1] 100

  Threshold         R2            P      BETA       SE
1  0.000001 0.08270392 0.0003606411 -13.71551 3.748008
